In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeRegressor,  DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from mlxtend.plotting import plot_decision_regions

from sklearn.metrics import roc_auc_score, classification_report, f1_score

from sklearn.decomposition import PCA

/home/w_lander/anaconda3/envs/dowgrade/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


1. Загрузите набор данных с помощью pandas.

2. Разделите данные на обучающую и тестовую выборки.

3. Импортируйте класс DecisionTreeClassifier из библиотеки sklearn.

4. Создайте экземпляр класса DecisionTreeClassifier и обучите его на обучающей выборке.

5. Оцените качество модели на тестовой выборке с помощью метрик, таких как точность (accuracy), полнота (recall), точность (precision) и F1-мера (F1-score).

6. Визуализируйте построенное дерево для анализа его структуры и принятия решений.

In [2]:
df = pd.read_csv('data/telecom_churn.csv')
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   3333 non-null   object 
 1   Account length          3333 non-null   int64  
 2   Area code               3333 non-null   int64  
 3   International plan      3333 non-null   object 
 4   Voice mail plan         3333 non-null   object 
 5   Number vmail messages   3333 non-null   int64  
 6   Total day minutes       3333 non-null   float64
 7   Total day calls         3333 non-null   int64  
 8   Total day charge        3333 non-null   float64
 9   Total eve minutes       3333 non-null   float64
 10  Total eve calls         3333 non-null   int64  
 11  Total eve charge        3333 non-null   float64
 12  Total night minutes     3333 non-null   float64
 13  Total night calls       3333 non-null   int64  
 14  Total night charge      3333 non-null   

In [4]:
df.isna().sum()

State                     0
Account length            0
Area code                 0
International plan        0
Voice mail plan           0
Number vmail messages     0
Total day minutes         0
Total day calls           0
Total day charge          0
Total eve minutes         0
Total eve calls           0
Total eve charge          0
Total night minutes       0
Total night calls         0
Total night charge        0
Total intl minutes        0
Total intl calls          0
Total intl charge         0
Customer service calls    0
Churn                     0
dtype: int64

In [5]:
df.head(2)

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.7,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.7,1,False


In [6]:
df = df.drop('State', axis=1)

In [7]:
mapping = {'No': 0, 'Yes': 1}
col_map = ['International plan', 'Voice mail plan']

for item in col_map:
    if df[item].unique()[1:] in ('Yes', 'No'):
        df[item] = df[item].map(mapping)

In [8]:
df['Churn'] = df['Churn'].replace({False: 0, True: 1})

/tmp/ipykernel_38097/3547494869.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Churn'] = df['Churn'].replace({False: 0, True: 1})


In [9]:
df['Churn'].value_counts()

Churn
0    2850
1     483
Name: count, dtype: int64

In [10]:
# Дисбаланс классов

### Построение модели

In [11]:
X = df.drop('Churn', axis=1)
Y = df['Churn']

X_scaled = pd.DataFrame(MinMaxScaler().fit_transform(X), columns=X.columns)
#X_scaled = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

In [12]:
X_train.shape,  X_test.shape

((2333, 18), (1000, 18))

In [13]:
models = []
predicts = []

In [14]:
model_lof_reg = LogisticRegression(class_weight='balanced')
model_lof_reg.fit(X_train, y_train)

pred_lof_reg = model_lof_reg.predict(X_test)
print('TEST')
print(classification_report(y_test, pred_lof_reg))

models.append(model_lof_reg)
predicts.append(pred_lof_reg)

TEST
              precision    recall  f1-score   support

           0       0.96      0.78      0.86       857
           1       0.38      0.80      0.51       143

    accuracy                           0.78      1000
   macro avg       0.67      0.79      0.69      1000
weighted avg       0.88      0.78      0.81      1000



In [15]:
model_svc = SVC(class_weight='balanced', kernel='linear')
model_svc.fit(X_train, y_train)

pred_svc = model_svc.predict(X_test)
print('TEST')
print(classification_report(y_test, pred_svc))

models.append(model_svc)
predicts.append(pred_svc)

TEST
              precision    recall  f1-score   support

           0       0.96      0.77      0.85       857
           1       0.37      0.80      0.50       143

    accuracy                           0.77      1000
   macro avg       0.66      0.79      0.68      1000
weighted avg       0.87      0.77      0.80      1000



In [16]:
model_tree = DecisionTreeClassifier()
model_tree.fit(X_train, y_train)

pred_tree = model_tree.predict(X_test)
print('TEST')
print(classification_report(y_test, pred_tree))

models.append(model_tree)
predicts.append(pred_tree)

TEST
              precision    recall  f1-score   support

           0       0.95      0.94      0.95       857
           1       0.67      0.73      0.70       143

    accuracy                           0.91      1000
   macro avg       0.81      0.84      0.83      1000
weighted avg       0.91      0.91      0.91      1000



In [17]:
score = []

for idx, model, in enumerate(models):

    model_split = str(model).split('(')
    
    score.append(
            {
             'model': model_split[0],
             'roc_auc_score': roc_auc_score(y_test, predicts[idx]) 
            }
    )

In [18]:
result = pd.DataFrame(score)
result

,model,roc_auc_score
0,LogisticRegression,0.790663
1,SVC,0.786579
2,DecisionTreeClassifier,0.837378


In [19]:
# DecisionTreeClassifier показала наилучший результат

### Алгоритмам на основе деревьев, такие как деревья решений, случайные леса и градиентный бустинг, нечувствительны к величине переменных. Поэтому стандартизация не требуется